In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from utils import *

In [3]:
dataframes = read_csv("./datasets", 'csv')

In [4]:
res = prepare_dataframes(dataframes)

  0%|          | 0/3 [00:00<?, ?it/s]

out_hum
pressure
out_temp


In [5]:
df = get_labels(res, '2024-02-13')

In [6]:
df['rain_1h'] = df['rain_1h'].fillna(0)
df['snow_1h'] = df['snow_1h'].fillna(0)
df['total percipitation'] = df['rain_1h'] + df['snow_1h']

In [7]:
df['last_changed_count'] = df.groupby('last_changed')['last_changed'].transform('count')
df = df.drop_duplicates(subset=['last_changed'])
df = df.loc[:, ['last_changed', 'pressure', 'out_temp', 'out_hum', 'total percipitation']]
df = df.reset_index(drop=True)

## Tabela 1 (time, P, T, H, R)

In [9]:
df

,last_changed,pressure,out_temp,out_hum,total percipitation
0,2023-09-20 00:00:00+00:00,986.000000,15.300000,76.000000,0.00
1,2023-09-20 01:00:00+00:00,985.428571,14.528571,78.900000,0.00
2,2023-09-20 02:00:00+00:00,985.760000,14.212500,79.500000,0.00
3,2023-09-20 03:00:00+00:00,985.280000,14.041667,81.300000,0.00
4,2023-09-20 04:00:00+00:00,985.666667,13.833333,83.333333,0.25
...,...,...,...,...,...
3499,2024-02-12 19:00:00+00:00,974.777778,7.220000,85.166667,0.00
3500,2024-02-12 20:00:00+00:00,975.846154,7.020000,84.933333,0.00
3501,2024-02-12 21:00:00+00:00,976.846154,6.566667,84.033333,0.00
3502,2024-02-12 22:00:00+00:00,976.521739,6.370000,83.873913,0.00


In [10]:
df.value_counts('total percipitation')

total percipitation
0.00     2666
0.25      166
0.51      149
0.38       37
1.02       29
         ... 
1.53        1
1.59        1
1.61        1
1.66        1
30.99       1
Name: count, Length: 119, dtype: int64

## Tabela 2 (time, 0, -1, -2, -3...)

In [11]:
df2 = generate_data(df, 5)
df2

,time,TP t+1,Press t0,Temp t0,Hum t0,TP t0,Press t-1,Temp t-1,Hum t-1,TP t-1,...,Hum t-3,TP t-3,Press t-4,Temp t-4,Hum t-4,TP t-4,Press t-5,Temp t-5,Hum t-5,TP t-5
0,2023-09-20 05:00:00+00:00,0.0,986.217391,13.966667,83.366667,0.0,985.666667,13.833333,83.333333,0.25,...,79.500000,0.00,985.428571,14.528571,78.900000,0.00,986.000000,15.300000,76.000000,0.00
1,2023-09-20 06:00:00+00:00,0.0,986.478261,14.640000,80.100000,0.0,986.217391,13.966667,83.366667,0.00,...,81.300000,0.00,985.760000,14.212500,79.500000,0.00,985.428571,14.528571,78.900000,0.00
2,2023-09-20 07:00:00+00:00,0.0,986.739130,15.480000,75.420000,0.0,986.478261,14.640000,80.100000,0.00,...,83.333333,0.25,985.280000,14.041667,81.300000,0.00,985.760000,14.212500,79.500000,0.00
3,2023-09-20 08:00:00+00:00,0.0,986.090909,17.000000,69.840000,0.0,986.739130,15.480000,75.420000,0.00,...,83.366667,0.00,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,81.300000,0.00
4,2023-09-20 09:00:00+00:00,0.0,986.200000,18.600000,61.920000,0.0,986.090909,17.000000,69.840000,0.00,...,80.100000,0.00,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,83.333333,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,2024-02-12 18:00:00+00:00,0.0,974.642857,7.371429,84.830000,0.0,973.500000,7.540000,84.800000,0.00,...,83.640000,0.00,971.652174,7.860000,85.928571,0.00,971.130435,7.955556,86.000000,0.00
3494,2024-02-12 19:00:00+00:00,0.0,974.777778,7.220000,85.166667,0.0,974.642857,7.371429,84.830000,0.00,...,83.020000,0.00,972.384615,7.877778,83.640000,0.00,971.652174,7.860000,85.928571,0.00
3495,2024-02-12 20:00:00+00:00,0.0,975.846154,7.020000,84.933333,0.0,974.777778,7.220000,85.166667,0.00,...,84.800000,0.00,973.222222,7.754545,83.020000,0.00,972.384615,7.877778,83.640000,0.00
3496,2024-02-12 21:00:00+00:00,0.0,976.846154,6.566667,84.033333,0.0,975.846154,7.020000,84.933333,0.00,...,84.830000,0.00,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,83.020000,0.00


In [12]:
X = df2.iloc[:,2:]
y = df2.iloc[:,1:2]

In [13]:
X

,Press t0,Temp t0,Hum t0,TP t0,Press t-1,Temp t-1,Hum t-1,TP t-1,Press t-2,Temp t-2,...,Hum t-3,TP t-3,Press t-4,Temp t-4,Hum t-4,TP t-4,Press t-5,Temp t-5,Hum t-5,TP t-5
0,986.217391,13.966667,83.366667,0.0,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,...,79.500000,0.00,985.428571,14.528571,78.900000,0.00,986.000000,15.300000,76.000000,0.00
1,986.478261,14.640000,80.100000,0.0,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,...,81.300000,0.00,985.760000,14.212500,79.500000,0.00,985.428571,14.528571,78.900000,0.00
2,986.739130,15.480000,75.420000,0.0,986.478261,14.640000,80.100000,0.00,986.217391,13.966667,...,83.333333,0.25,985.280000,14.041667,81.300000,0.00,985.760000,14.212500,79.500000,0.00
3,986.090909,17.000000,69.840000,0.0,986.739130,15.480000,75.420000,0.00,986.478261,14.640000,...,83.366667,0.00,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,81.300000,0.00
4,986.200000,18.600000,61.920000,0.0,986.090909,17.000000,69.840000,0.00,986.739130,15.480000,...,80.100000,0.00,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,83.333333,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,974.642857,7.371429,84.830000,0.0,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,...,83.640000,0.00,971.652174,7.860000,85.928571,0.00,971.130435,7.955556,86.000000,0.00
3494,974.777778,7.220000,85.166667,0.0,974.642857,7.371429,84.830000,0.00,973.500000,7.540000,...,83.020000,0.00,972.384615,7.877778,83.640000,0.00,971.652174,7.860000,85.928571,0.00
3495,975.846154,7.020000,84.933333,0.0,974.777778,7.220000,85.166667,0.00,974.642857,7.371429,...,84.800000,0.00,973.222222,7.754545,83.020000,0.00,972.384615,7.877778,83.640000,0.00
3496,976.846154,6.566667,84.033333,0.0,975.846154,7.020000,84.933333,0.00,974.777778,7.220000,...,84.830000,0.00,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,83.020000,0.00


In [14]:
y

,TP t+1
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3493,0.0
3494,0.0
3495,0.0
3496,0.0


In [15]:
kfold = KFold(n_splits = 5)

classifiers = [
    LinearRegression(),   
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        #'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        #'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_lr_1 = pd.DataFrame(results_list)

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
df_lr_1

,Classifier,r2,neg_mean_squared_error,neg_mean_absolute_error
0,LinearRegression,0.041387,-0.931972,-0.271077


## LR coef

In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
)

In [31]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

In [47]:
print('Wzór funkcji: y = {} * x1 + {} * x2 + {} * x3 + {} * x4 + {} * x5 + {}* x6 + {} * x7 + {} * x8 + {} * x9 + {}* x10 + {} * x11 + {} * x12 + {} * x13 + {}* x14 + {} * x15 + {} * x16 + {} * x17 + {}* x18 + {} * x19 + {} * x20'.format(
    lr.coef_[0][0],
    lr.coef_[0][1],
    lr.coef_[0][2],
    lr.coef_[0][3],
    lr.coef_[0][4],
    lr.coef_[0][5],
    lr.coef_[0][6],
    lr.coef_[0][7],
    lr.coef_[0][8],
    lr.coef_[0][9],
    lr.coef_[0][10],
    lr.coef_[0][11],
    lr.coef_[0][12],
    lr.coef_[0][13],
    lr.coef_[0][14],
    lr.coef_[0][15],
    lr.coef_[0][16],
    lr.coef_[0][17],
    lr.coef_[0][18],
    lr.coef_[0][19],
    lr.coef_[0][20],
    lr.intercept_))

Wzór funkcji: y = 0.08536698135348043 * x1 + 0.1364605347923451 * x2 + 0.0313905974120792 * x3 + 0.20203291950043728 * x4 + -0.06417027184819155 * x5 + -0.3051310438943063* x6 + -0.03096022224985376 * x7 + 0.014798517388148915 * x8 + -0.06611485637597231 * x9 + 0.2247331486919702* x10 + 0.0023467897452058096 * x11 + 0.019617238533637574 * x12 + -0.006242354517986123 * x13 + -0.06812878049782757* x14 + -0.004106339902391713 * x15 + 0.016100974870865786 * x16 + 0.02887959707156466 * x17 + 0.005125923675986886* x18 + 0.008886748662687962 * x19 + 0.0010199123097958335 * x20


In [45]:
lr.coef_

array([[ 0.08536698,  0.13646053,  0.0313906 ,  0.20203292, -0.06417027,
        -0.30513104, -0.03096022,  0.01479852, -0.06611486,  0.22473315,
         0.00234679,  0.01961724, -0.00624235, -0.06812878, -0.00410634,
         0.01610097,  0.0288796 ,  0.00512592,  0.00888675,  0.00101991,
         0.02283603,  0.01218289, -0.00057419,  0.01479573]])